# Kafka Data Consumer

## I/ Objectifs du notebook

## II/ Mécanisme de collecte en streaming des données


## III/ Mécanismes de validation et corrections des données

## IV/ Mécanismes de mise en base des données

In [33]:
import numpy as np
import sys
sys.path.append("../TimeSeriesTools")

In [34]:
import mongodb_utils
db_host= 'localhost'
port = '27018'
db_name='TimeSeriesBench'
client = mongodb_utils.mongodb_connect(db_host, port)
db = client[db_name]

Trying to connect to MongoDB server: localhost on port: 27018


### A/ Mise en base du schéma des données

#### 1) Cas test SmartGrid

In [19]:
coll_name='SmartGridCryolite20190101OneDay'
scheme = { 'name' : 'SmartGrid',
           'value' : {"_id":{'required': True},
                  "timestamp":{'type': 'string','required': True,'empty': False},
                  "tagname":{'type': 'string','required': True,'empty': False},
                  "value":{'type': 'string','required': True,'empty': False},
                  "quality":{'type': 'string','required': True,'empty': False}} }


In [35]:
db.list_collection_names()

['WindPropLacqOneDayP25',
 'schemes',
 'WindPropLacqOneDay',
 'WindPropLacqOneDayV2']

In [21]:
scheme_collection = db['schemes']

In [22]:
scheme_id = scheme_collection.insert_one(scheme)

In [23]:
scheme_id

### 2) Cas test WindProp

In [36]:
coll_name='WindPropLacqOneDayV3'

In [37]:
client.list_database_names()

['TimeSeriesBench', 'admin', 'config', 'local', 'test']

In [38]:
db.list_collection_names()

['WindPropLacqOneDayP25',
 'schemes',
 'WindPropLacqOneDay',
 'WindPropLacqOneDayV2']

In [39]:
scheme_collection = db['schemes']

In [43]:
scheme =  { 'name' : 'WindPropLacqV3',
            'value' : {
    "_id":{'required': True},
    "Heure":{'type': 'string','required': True,'empty': False},
    "Temps écoulé":{'type': 'string','required': True,'empty': False},
    '4069 state':{'type': 'string','required': True,'empty': False},
    'Battery voltage':{'type': 'string','required': True,'empty': False},
    'Flow SP':{'type': 'string','required': True,'empty': False},
    'Unit Code':{'type': 'string','required': True,'empty': False},
    'Flow M':{'type': 'string','required': True,'empty': False},
    'Pressure1':{'type': 'string','required': True,'empty': False},
    'Temperature1':{'type': 'string','required': True,'empty': False},
    "Latitude":{'type': 'string','required': True,'empty': False},
    "Longitude":{'type': 'string','required': True,'empty': False},
    "Altitude":{'type': 'string','required': True,'empty': False},
    "Head_Rel_True North":{'type': 'string','required': True,'empty': False},
    "Pressure2":{'type': 'string','required': True,'empty': False},
    "Temperature2":{'type': 'string','required': True,'empty': False},
    "Humidity":{'type': 'string','required': True,'empty': False},
    "MDA Wnd Dir":{'type': 'string','required': True,'empty': False},
    "MDA Wnd Speed":{'type': 'string','required': True,'empty': False},
    "MWD Wind Dir":{'type': 'string','required': True,'empty': False},
    "MWD Wind Speed":{'type': 'string','required': True,'empty': False},
    'Gaz Concentration 1':{'type': 'string','required': True,'empty': False},
    'Gaz Concentration 2':{'type': 'string','required': True,'empty': False},
    'Gaz Concentration 3':{'type': 'string','required': True,'empty': False},
    'Gaz Concentration 4':{'type': 'string','required': True,'empty': False},
    'SPA 1':{'type': 'string','required': True,'empty': False},
    'SPA 2':{'type': 'string','required': True,'empty': False},
    'Cellule Photo':{'type': 'string','required': True,'empty': False},
    'Temperature3':{'type': 'string','required': True,'empty': False},
    'Pressure3':{'type': 'string','required': True,'empty': False},
    'Flow MassFlow 1':{'type': 'string','required': True,'empty': False},
    'NOTUSED Flow MassFlow 2':{'type': 'string','required': True,'empty': False},
    'Flow,Humidity':{'type': 'string','required': True,'empty': False},
    'Test':{'type': 'string','required': True,'empty': False},
    'Details':{'type': 'string','required': True,'empty': False},
    'SPA 3':{'type': 'string','required': True,'empty': False},
    'SPA 4':{'type': 'string','required': True,'empty': False},
    "CavityPressure":{'type': 'string','required': True,'empty': False},    
    "CavityTemp":{'type': 'string','required': True,'empty': False},
    "CH4":{'type': 'string','required': True,'empty': False},
    "CH4_dry":{'type': 'string','required': True,'empty': False},
    "C2H6":{'type': 'string','required': True,'empty': False},
    "C2H6_dry":{'type': 'string','required': True,'empty': False},
    "13CH4":{'type': 'string','required': True,'empty': False},
    "H2O":{'type': 'string','required': True,'empty': False},
    "CO2":{'type': 'string','required': True,'empty': False},
    "C2C1Ratio":{'type': 'string','required': True,'empty': False},
    "Delta_iCH4_Raw":{'type': 'string','required': True,'empty': False},
    "HP_Delta_iCH4_30s":{'type': 'string','required': True,'empty': False},
    "HP_Delta_iCH4_2min":{'type': 'string','required': True,'empty': False},
    "HP_Delta_iCH4_5min":{'type': 'string','required': True,'empty': False}
    } }

In [44]:
scheme_id = scheme_collection.insert_one(scheme)

In [45]:
scheme_id